## Loading pre-trained sense vectors 

To test with word sense embeddings you can use a pretrained model (sense vectors and sense probabilities). These sense         vectors were induced from Wikipedia using word2vec similarities between words in ego-networks. Sense probabilities are       stored in a separate file which is located next to the file with sense vectors. 

In [2]:
import sensegram
sense_vectors_fpath = "model/dewiki.txt.clusters.minsize5-1000-sum-score-20.sense_vectors"
sv = sensegram.SenseGram.load_word2vec_format(sense_vectors_fpath, binary=False)

## Getting the list of senses of a word 

Probabilities of senses will be loaded automatically if placed in the same folder as sense vectors and named according to the  same scheme as our pretrained files.

To examine how many senses were learned for a word call `get_senses` funcion:

In [5]:
word = "Hund"
sv.get_senses(word)

[('Hund#1', 0.976526), ('Hund#2', 0.023474)]

## Sense aware nearest neighbors

The function returns a list of sense names with probabilities for each sense. As one can see, our model has learned two senses for the word "ключ".

To understand which word sense is represented with a sense vector use `most_similar` function:


In [6]:
word = "Hund"
for sense_id, prob in sv.get_senses(word):
    print(sense_id)
    print("="*20)
    for rsense_id, sim in sv.wv.most_similar(sense_id):
        print("{} {:f}".format(rsense_id, sim))
    print("\n")

Hund#1
Papagei#1 0.948971
Pudel#1 0.948960
Mops#1 0.943027
Dackel#1 0.938603
Teddybär#1 0.934012
Zwilling#1 0.930496
Kater#1 0.929611
Kanarienvogel#1 0.927734
Bernhardiner#1 0.926941
Kauz#1 0.922378


Hund#2
guter_Kamerad#1 0.942419
bunter_Abend#1 0.939721
Petschorin#1 0.938157
hinzukommender#1 0.937122
geplantes_Comeback#1 0.936010
schicksalhafter#1 0.935064
Grosse_Erweckung#3 0.934708
armer_Hund#1 0.934650
Alois_Windisch#2 0.934103
eingezogener_spitzbogiger#1 0.933642




## Word sense disambiguation: loading word embeddings

To use our word sense disambiguation mechanism you also need word vectors or context vectors, depending on the dismabiguation  strategy. Those word are located in the ``model`` directory and has the extension ``.vectors``.

Our WSD mechanism is based on word similarities (`sim`) and requires word vectors to represent context words. In following we provide a disambiguation example using similarity strategy.

First, load word vectors using gensim library:


In [7]:
from gensim.models import KeyedVectors
word_vectors_fpath = "model/dewiki.txt.word_vectors"
wv = KeyedVectors.load_word2vec_format(word_vectors_fpath, binary=False, unicode_errors="ignore")

Then initialise the WSD object with sense and word vectors:

In [9]:
from wsd import WSD
wsd_model = WSD(sv, wv, window=5, method='sim', filter_ctx=3)

Disambiguation method: sim
Filter context: f = 3


The settings have the following meaning: it will extract at most `window`*2 words around the target word from the  sentence as context and it will use only three most discriminative context words for disambiguation.

Now you can disambiguate the word "table" in the sentence "They bought a table and chairs for kitchen" using       `dis_text` function. As input it takes a sentence with space separated tokens, a target word, and start/end indices of the target word in the given sentence.


In [10]:
word = "Hund"
context = "Die beste Voraussetzung für die Hund-Katze-Freundschaft ist, dass keiner von beiden in der Vergangenheit unangenehme Erlebnisse mit der anderen Gattung hatte. Am einfachsten ist die ungleiche WG, wenn sich zwei Jungtiere ein Zuhause teilen. Bei erwachsenen Tieren ist es einfacher, wenn sich Miezi in Bellos Haushalt einnistet – nicht umgekehrt, da Hunde Rudeltiere sind. Damit ein Hund das Kätzchen aber auch als Rudelmitglied sieht und nicht als Futter sollten ein paar Regeln beachtet werden"
wsd_model.dis_text(context, word, 0, 4)

('Hund#1', [0.11995293847479865, -0.10629601676025206])

# SDEWaC corpus

In [44]:
import sensegram
from wsd import WSD
from gensim.models import KeyedVectors


# Input data and paths
sense_vectors_fpath = "model/sdewac-v3.corpus.clusters.minsize5-1000-sum-score-20.sense_vectors"
word_vectors_fpath = "model/sdewac-v3.corpus.word_vectors"
context_words_max = 3 # change this paramters to 1, 2, 5, 10, 15, 20 : it may improve the results
context_window_size = 5 # this parameters can be also changed during experiments 
word = "Maus"
context = "Die Maus ist ein Eingabegerät (Befehlsgeber) bei Computern. Der allererste Prototyp wurde 1963 nach Zeichnungen von Douglas C. Engelbart gebaut; seit Mitte der 1980er Jahre bildet die Maus für fast alle Computertätigkeiten zusammen mit dem Monitor und der Tastatur eine der wichtigsten Mensch-Maschine-Schnittstellen. Die Entwicklung grafischer Benutzeroberflächen hat die Computermaus zu einem heute praktisch an jedem Desktop-PC verfügbaren Standardeingabegerät gemacht."
ignore_case = True

# Load models (takes long time)
sv = sensegram.SenseGram.load_word2vec_format(sense_vectors_fpath, binary=False)
wv = KeyedVectors.load_word2vec_format(word_vectors_fpath, binary=False, unicode_errors="ignore")

# Play with the model (is quick)
print("Probabilities of the senses:\n{}\n\n".format(sv.get_senses(word, ignore_case=ignore_case)))

for sense_id, prob in sv.get_senses(word, ignore_case=ignore_case):
    print(sense_id)
    print("="*20)
    for rsense_id, sim in sv.wv.most_similar(sense_id):
        print("{} {:f}".format(rsense_id, sim))
    print("\n")

# Disambiguate a word in a context
wsd_model = WSD(sv, wv, window=context_window_size, lang="de",
                filter_ctx=context_words_max, ignore_case=ignore_case)    
print(wsd_model.disambiguate(context, word))

Probabilities of the senses:
[('Maus#1', 0.885167), ('Maus#2', 0.114833), ('maus#1', 1.0)]


Maus#1
Wanze#1 0.941433
Mausefalle#1 0.911212
Libelle#1 0.908983
Puppe#2 0.908126
Kakerlake#1 0.907264
Ratte#1 0.904579
Spinne#1 0.904541
Garnele#1 0.904265
Biene#1 0.902711
Schnecke#1 0.902516


Maus#2
Cursortasten#1 0.972510
Pfeiltasten#1 0.964843
Richtungstasten#1 0.959181
Cursor_Tasten#1 0.955643
linken_Maustaste#1 0.945940
Tab_Taste#1 0.944255
gedrückter_Maustaste#2 0.937142
rechten_Maustaste#1 0.935822
Mausbewegung#1 0.935422
Sprungtaste#2 0.933055


maus#1
hp#2 0.970098
gedichte#1 0.954409
galerie#1 0.951155
meinem_bild#3 0.945498
fotografie#1 0.941011
alex#1 0.939576
wolf#1 0.935995
ds#1 0.934641
nachricht#3 0.934026
anfrage#1 0.933526


Disambiguation method: sim
Filter context: f = 3
Context words:
Eingabegerät	0.394
('Maus#2', [0.34084959179662255, 0.52808930616226313, 0.13438711940973358])


# Word sense induction exepriment for the Russian language

In [1]:
import sensegram
from wsd import WSD
from gensim.models import KeyedVectors


# Input data and paths
sense_vectors_fpath = "model/wikipedia-ru-2018.txt.clusters.minsize5-1000-sum-score-20.sense_vectors"
word_vectors_fpath = "model/wikipedia-ru-2018.txt.word_vectors"
context_words_max = 3 # change this paramters to 1, 2, 5, 10, 15, 20 : it may improve the results
context_window_size = 5 # this parameters can be also changed during experiments 
word = "ключ"
context = "Ключ — это секретная информация, используемая криптографическим алгоритмом при зашифровании/расшифровании сообщений, постановке и проверке цифровой подписи, вычислении кодов аутентичности (MAC). При использовании одного и того же алгоритма результат шифрования зависит от ключа. Для современных алгоритмов сильной криптографии утрата ключа приводит к практической невозможности расшифровать информацию."
ignore_case = True
lang = "ru" # to filter out stopwords

# Load models (takes long time)
sv = sensegram.SenseGram.load_word2vec_format(sense_vectors_fpath, binary=False)
wv = KeyedVectors.load_word2vec_format(word_vectors_fpath, binary=False, unicode_errors="ignore")

# Play with the model (is quick)
print("Probabilities of the senses:\n{}\n\n".format(sv.get_senses(word, ignore_case=ignore_case)))

for sense_id, prob in sv.get_senses(word, ignore_case=ignore_case):
    print(sense_id)
    print("="*20)
    for rsense_id, sim in sv.wv.most_similar(sense_id):
        print("{} {:f}".format(rsense_id, sim))
    print("\n")

# Disambiguate a word in a context
wsd_model = WSD(sv, wv, window=context_window_size, lang=lang,
                filter_ctx=context_words_max, ignore_case=ignore_case)    
print(wsd_model.disambiguate(context, word))

Probabilities of the senses:
[('ключ#1', 0.33494), ('ключ#2', 0.279518), ('ключ#3', 0.248193), ('ключ#4', 0.125301), ('ключ#5', 0.012048), ('Ключ#1', 0.29471), ('Ключ#2', 0.289673), ('Ключ#3', 0.216625), ('Ключ#4', 0.198992)]


ключ#1
Ключ#1 0.991653
колодец#2 0.915456
ЯМА#2 0.899536
Яма#2 0.899536
яма#2 0.899536
ищущий#2 0.897314
Ищущий#2 0.897314
Иллюзий#4 0.896384
Змеиной#3 0.894822
змеиной#4 0.894822


ключ#2
Ключ#2 0.995825
дубликат#1 0.942747
Дубликат#1 0.942747
чемоданчик#1 0.935276
брелок#2 0.929783
Брелок#2 0.929783
Кошелёк#1 0.926213
Кулон#1 0.923792
кулон#1 0.923792
Кейс#2 0.923318


ключ#3
симметричный_ключ#1 0.948021
ключевой_поток#1 0.945735
зашифрованный_текст#1 0.944024
сеансовый_ключ#1 0.944022
ПИН_код#1 0.939620
пин_код#1 0.939620
шифротекст#2 0.938751
шифротекст_formula_#1 0.937948
Секретный_ключ#1 0.936267
секретный_ключ#1 0.936267


ключ#4
Ключ#4 0.952905
яр#1 0.952719
овраг#1 0.951952
Яр#2 0.951806
порог#2 0.951373
Порог#2 0.951373
ЯР#2 0.949762
хуторок#2 0.945221

In [27]:
from pandas import read_csv 


# you can download the WSI evaluation dataset with 'git clone https://github.com/nlpub/russe-wsi-kit.git'
wikiwiki_fpath = "../russe-wsi-kit/data/main/wiki-wiki/test.csv"
activedict_fpath = "../russe-wsi-kit/data/main/active-dict/test.csv"
btsrnc_fpath = "../russe-wsi-kit/data/main/bts-rnc/test.csv"

def evaluate(dataset_fpath):
    output_fpath = dataset_fpath + ".pred"

    df = read_csv(wikiwiki_fpath, sep="\t", encoding="utf-8")

    for i, row in df.iterrows():
        sense_id, _ = wsd_model.disambiguate(row.context, row.word)
        df.loc[i, "predict_sense_id"] = sense_id

    df.to_csv(output_fpath, sep="\t", encoding="utf-8")
    print("Output:", output_fpath)
    
    return output_fpath


evaluate(activedict_fpath)
evaluate(btsrnc_fpath)
evaluate(wikiwiki_fpath)

Context words:
банка	0.655
здание	0.156
bank	0.104
Context words:
банки	0.381
времена	0.109
Context words:
банка	0.655
национального	0.315
украины	0.157
Context words:
почтового	0.473
зданием	0.212
является	0.211
Context words:
учреждения	0.349
ведения	0.131
функционировать	0.073
Context words:
банка	0.655
подводный	0.075
риф	0.046
Context words:
рейхсмарки	0.196
эмиссионного	0.190
обменены	0.128
Context words:
директоров	0.384
президент	0.273
возглавляет	0.168
Context words:
банка	0.655
лена	0.286
экспедиции	0.134
Context words:
конвенции	0.160
настоящее	0.145
находится	0.102
Context words:
денег	0.191
нумизматический	0.157
bank	0.104
Context words:
банк	0.423
денег	0.191
музей	0.100
Context words:
платежа	0.313
билетов	0.246
законным	0.183
Context words:
банка	0.655
шельфовая	0.113
отмель	0.066
Context words:
небольшого	0.273
отверстия	0.137
жидкое	0.129
Context words:
банка	0.655
тара	0.091
пивная	0.055
Context words:
торгового	0.505
дома	0.220
формах	0.137
Context words:
календаря	

Context words:
анимационный	0.479
фильм	0.345
собаки	0.326
Context words:
обыкновенная	0.609
лесах	0.465
особенностями	0.353
Context words:
белка	0.803
грызунов	0.460
alleni	0.371
Context words:
лисья	0.632
тела	0.401
составляет	0.242
Context words:
запуск	0.552
спутник	0.542
июля	0.188
Context words:
зубов	0.557
продольные	0.255
желобки	0.237
Context words:
белка	0.803
капская	0.725
земляная	0.337
Context words:
белки	0.681
самки	0.434
самцы	0.418
Context words:
кочующая	0.452
идет	0.240
длительные	0.231
Context words:
собака	0.466
цирковая	0.450
белая	0.418
Context words:
тосковать	0.214
надоедать	0.105
стала	0.095
Context words:
белков	0.951
альфа	0.758
доменов	0.734
Context words:
мотивам	0.425
собаки	0.326
стрелка	0.249
Context words:
семейка	0.698
озорная	0.510
стрелка	0.249
Context words:
членов	0.374
мгновение	0.264
экипажа	0.257
Context words:
семейка	0.698
озорная	0.510
мультсериала	0.286
Context words:
обыкновенная	0.609
белку	0.445
конкурентка	0.314
Context words:
белка	0.8

Context words:
ущелье	0.506
высота	0.322
моджахедов	0.311
Context words:
фильм	0.339
парафраз	0.331
беда	0.248
Context words:
районе	0.473
расположен	0.354
храм	0.330
Context words:
горе	0.642
битва	0.177
полулегендарная	0.100
Context words:
камни	0.356
обелиска	0.349
автором	0.247
Context words:
горе	0.642
сражение	0.218
битва	0.177
Context words:
фильм	0.339
радости	0.323
Context words:
берегов	0.308
ров	0.295
меж	0.152
Context words:
горе	0.642
драматургом	0.464
трагедия	0.412
Context words:
протестантов	0.195
баланс	0.184
европе	0.158
Context words:
горе	0.642
вышка	0.308
комплекс	0.241
Context words:
покрова	0.308
церковь	0.292
пресвятой	0.162
Context words:
горе	0.642
ума	0.444
название	0.153
Context words:
горе	0.642
повесть	0.470
Context words:
горе	0.642
ума	0.444
Context words:
кукольный	0.526
мультфильм	0.329
советский	0.157
Context words:
горе	0.642
мемориал	0.260
синагога	0.244
Context words:
горе	0.642
храм	0.330
православный	0.072
Context words:
сражение	0.218
битва	0.17

проводника	0.424
Context words:
людей	0.722
ангел	0.625
колец	0.557
Context words:
проводников	0.481
старается	0.392
поэтому	0.333
Context words:
мечом	0.620
пронзил	0.492
побеге	0.388
Context words:
море	0.412
начиная	0.135
Context words:
проводников	0.481
аниме	0.436
исследовательского	0.383
Context words:
отвергает	0.491
переселение	0.339
христианство	0.294
Context words:
проводников	0.481
воли	0.330
друг	0.280
Context words:
вылечить	0.397
чумы	0.379
продавший	0.182
Context words:
network	0.342
anime	0.292
news	0.266
Context words:
меч	0.736
огненный	0.596
оружие	0.452
Context words:
лидер	0.435
боец	0.389
друг	0.280
Context words:
октября	0.267
фильмом	0.266
вышел	0.171
Context words:
эпизода	0.475
телешоу	0.333
смотреть	0.319
Context words:
одиночку	0.487
троих	0.471
вторжения	0.362
Context words:
пустых	0.688
силы	0.377
проводники	0.308
Context words:
победить	0.621
чувствовать	0.378
видеть	0.365
Context words:
догадался	0.362
двойственности	0.319
лабораторию	0.273
Context words

государственного	0.328
здания	0.252
Context words:
национального	0.315
образное	0.183
н	0.175
Context words:
китайского	0.140
дизайн	0.126
башня	0.054
Context words:
министерства	0.338
счета	0.311
организациям	0.241
Context words:
корпорация	0.299
японская	0.125
разработала	0.084
Context words:
пко	0.305
здание	0.156
рыночная	0.114
Context words:
банка	0.655
офиса	0.602
головного	0.165
Context words:
представляющий	0.158
вулканического	0.070
подводную	0.066
Context words:
фонда	0.505
международного	0.399
валютного	0.395
Context words:
miller	0.201
genuine	0.147
говорящим	0.088
Context words:
банка	0.655
польского	0.129
рубль	0.087
Context words:
банка	0.655
эмиссионного	0.190
злотый	0.046
Context words:
банка	0.655
центр	0.151
культурный	0.086
Context words:
сая	0.177
меридиональном	0.170
северу	0.127
Context words:
почтового	0.473
ратуши	0.308
строительства	0.223
Context words:
компания	0.254
должного	0.148
нового	0.138
Context words:
невозможно	0.112
подвижные	0.092
возникла	0.086
Co

...

брат	0.452
Context words:
граф	0.871
ок	0.391
vii	0.130
Context words:
граф	0.871
ок	0.391
iv	0.251
Context words:
граф	0.871
де	0.543
ум	0.405
Context words:
де	0.543
титул	0.299
воссоздан	0.114
Context words:
й	0.304
погиб	0.288
войны	0.218
Context words:
граф	0.871
изоморфный	0.426
своему	0.263
Context words:
графов	0.663
неизоморфных	0.561
кубических	0.513
Context words:
граф	0.871
другом	0.186
написании	0.035
Context words:
треугольников	0.626
m	0.446
любого	0.390
Context words:
титул	0.299
получил	0.221
упоминается	0.204
Context words:
графом	0.658
грея	0.588
точки	0.453
Context words:
k	0.567
связность	0.531
вершин	0.509
Context words:
графов	0.663
графом	0.658
корневой	0.465
Context words:
принц	0.750
французский	0.409
мэнский	0.317
Context words:
граф	0.871
кент	0.406
й	0.304
Context words:
решетки	0.428
g	0.424
b	0.424
Context words:
графов	0.663
графом	0.658
пересечений	0.590
Context words:
граф	0.871
ы	0.200
известны	0.126
Context words:
коронации	0.452
свержения	0.354
титу

капитале	0.232
доли	0.210
определяется	0.054
Context words:
банка	0.655
директоров	0.384
директора	0.244
Context words:
контейнера	0.194
металла	0.152
абсолютно	0.140
Context words:
банк	0.423
эмиссионного	0.190
эвакуирован	0.152
Context words:
кредитных	0.287
инкассации	0.252
банкнот	0.220
Context words:
национального	0.315
отделение	0.181
украины	0.157
Context words:
характера	0.207
россии	0.170
приложении	0.162
Context words:
банка	0.655
активы	0.369
собственности	0.213
Context words:
фирменную	0.119
фасад	0.107
вывеску	0.103
Context words:
государственного	0.328
империи	0.148
настоящее	0.145
Context words:
банка	0.655
лена	0.286
расположена	0.080
Context words:
обязательства	0.289
пассивы	0.275
руководство	0.146
Context words:
правления	0.302
культурного	0.301
наследия	0.202
Context words:
банка	0.655
торгового	0.505
международного	0.399
Context words:
торгового	0.505
дома	0.220
дом	0.090
Context words:
банка	0.655
всемирного	0.259
шеф	0.207
Context words:
library	0.146
s	0.142
u	0

грызунов	0.460
рода	0.275
Context words:
белки	0.681
североамериканская	0.511
поскольку	0.320
Context words:
анимационный	0.479
фильм	0.345
собаки	0.326
Context words:
обыкновенная	0.609
лесах	0.465
особенностями	0.353
Context words:
белка	0.803
грызунов	0.460
alleni	0.371
Context words:
лисья	0.632
тела	0.401
составляет	0.242
Context words:
запуск	0.552
спутник	0.542
июля	0.188
Context words:
зубов	0.557
продольные	0.255
желобки	0.237
Context words:
белка	0.803
капская	0.725
земляная	0.337
Context words:
белки	0.681
самки	0.434
самцы	0.418
Context words:
кочующая	0.452
идет	0.240
длительные	0.231
Context words:
собака	0.466
цирковая	0.450
белая	0.418
Context words:
тосковать	0.214
надоедать	0.105
стала	0.095
Context words:
белков	0.951
альфа	0.758
доменов	0.734
Context words:
мотивам	0.425
собаки	0.326
стрелка	0.249
Context words:
семейка	0.698
озорная	0.510
стрелка	0.249
Context words:
членов	0.374
мгновение	0.264
экипажа	0.257
Context words:
семейка	0.698
озорная	0.510
мультсериала	

Context words:
граф	0.871
кубический	0.426
котором	0.183
Context words:
короля	0.620
падеж	0.321
conte	0.302
Context words:
треугольников	0.626
планарный	0.512
любой	0.402
Context words:
граф	0.871
является	0.326
рассматривать	0.227
Context words:
расстояний	0.528
конечный	0.497
единичных	0.493
Context words:
короля	0.620
падеж	0.321
conte	0.302
Context words:
граф	0.871
бургундский	0.536
бургундской	0.414
Context words:
граф	0.871
наборам	0.268
двойка	0.249
Context words:
twitter	0.367
имеют	0.344
возможность	0.236
Context words:
k	0.567
связность	0.531
вершин	0.509
Context words:
граф	0.871
кубическим	0.487
обхватом	0.451
Context words:
граф	0.871
битве	0.272
погибли	0.166
Context words:
графа	0.663
подграфы	0.536
пример	0.319
Context words:
графа	0.663
гиперкуба	0.654
точку	0.390
Context words:
граф	0.871
треугольник	0.503
любой	0.402
Context words:
гиперкуба	0.654
гамильтонов	0.505
клетка	0.436
Context words:
графа	0.663
брат	0.452
перси	0.250
Context words:
симметричное	0.516
влож

'../russe-wsi-kit/data/main/wiki-wiki/test.csv.pred'

In [25]:
df

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,440,банка,NaN,банка#1,"7-12, 36-41",здание банка китая ( макао ) здание банка кита...
1,441,банка,NaN,банка#1,"13-18, 201-206",трехлитровая банка во времена ссср такие банки...
2,442,банка,NaN,банка#1,"21-26, 58-63, 101-106, 263-268, 538-543",здание национального банка украины в здании на...
3,443,банка,NaN,банка#1,"134-139, 262-267, 345-350",зюдфиртель города эссен ( федеральная земля се...
4,444,банка,NaN,банка#1,"43-48, 300-305, 381-386, 521-526",ведения боевых действий полевые учреждения бан...
5,445,банка,NaN,банка#1,"11-16, 19-24, 88-93","вачусетт ( банка ) банка вачусетт , "" риф вачу..."
6,446,банка,NaN,Банка#1,"204-209, 292-297",— годах подпольной типографией союза вооруженн...
7,447,банка,NaN,Банка#1,"43-48, 331-336, 379-384, 557-562","директоров , который возглавляет президент бан..."
8,448,банка,NaN,банка#1,"284-289, 303-308",- электроходом обь в огромной африканско - ант...
9,449,банка,NaN,Банка#1,196-201,составлял около т . в году общая ориентировоч...
